In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns 
import pandas_profiling

C:\Anaconda3\lib\site-packages\matplotlib\__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [3]:
from scipy.stats import chi2
from scipy.stats import chi2_contingency

In [4]:
cd H:\Documents\Septimo semestre\Ciencia de datos aplicada\Taller estadistica bivariada

H:\Documents\Septimo semestre\Ciencia de datos aplicada\Taller estadistica bivariada


In [5]:
datos= pd.read_excel("tips.xls")

**1. Usted fue contratado para realizar el análisis estadístico de los datos recolectados. Realice
una inspección del data set y documente los hallazgos.**

In [6]:
pandas_profiling.ProfileReport(datos)

Number of variables,7
Number of observations,244
Total Missing (%),0.0%
Total size in memory,13.4 KiB
Average record size in memory,56.3 B
Numeric,3
Categorical,4
Date,0
Text (Unique),0
Rejected,0
Distinct count,4


El DataSet tiene 3 variables numéricas y 4 categóricas. No hay ningún valor faltante en los datos.  
En cuanto a las variables categóricas tenemos:  
Sexo: Podemos observar que entre los dos valores posibles, hombres y mujeres, se registra más pago de propinas por parte de los hombres.  
Día: Con esta podemos observar que el restaurante solo atiende 4 días de la semana, desde el jueves hasta el domingo.  
Fumador: Es un booleano (si y no) sobre la condición de fumador del que paga la cuenta.   
Tiempo: Como tiempo de comida el restaurante solo registra cena y almuerzo.  
Mientras que por las variables númericas:  
Tamaño: Registra la cantidad de personas en la mesa, los datos se distribuyen entre 1 y 6 personas. A grandes rasgos se puede observar que las personas que asisten al restaurante van en pareja.   
Propina: Podemos observar que las propinas se registran en dolares y están entre 1 y 10 dolares. Es una variable contínua, pues puede tomar cualquier valor real debido a los centavos. También podemos observar que hay datos repetidos en un 50%, por lo que se podría decir que hay personas que consideran dar la misma propina.  
Costo factura: Al igual que la propina se encuentra en dolares, tiene un valor mínimo de 3.07 dolares y un máximo de 50.81. Podemos observar que es un atributo completamente variable.

** 2. Realice un análisis descriptivo de la base de datos. Este debe contener al menos un diagrama
de dispersión, y un diagrama de cajas. ¿Qué hipótesis o reglas de negocio encuentra
congruente con este análisis?**

In [ ]:
plt.scatter()

**4. ¿Existe relación alguna entre el sexo y la propina?**

In [7]:
pd.value_counts(datos['sex'])

Male      157
Female     87
Name: sex, dtype: int64

Existen casi el doble de registros de propinas otorgadas por hombres, sin embargo eso no significa que por ser hombre exista una mayor probabilidad de que se de una propina, solo que la mayoría de los que pagan las cuentas suelen ser hombres.  
Se procede a realizar una prueba chi cuadrado que muestre si existe una dependencia entre las variables; sin embargo, primero se debe obtener el procentaje que dejan de propina las personas, basado en el valor de la cuenta; para luego convertir la variable de porcentaje (cuantitativa) en una variable cualitativa, que permita realizar una prueba chi cuadrado.  
Para estas conclusiones se va a asumir que una propina alta es superior al 10% del valor de la cuenta, teniendo en cuenta que una propina en Colombia debe rondar el 15% del valor de la cuenta.

In [8]:
propinasCategorizadas = datos
propinasCategorizadas['porcentaje'] = propinasCategorizadas['tip']/propinasCategorizadas['total_bill']
propinasCategorizadas['tipoP'] = np.where(propinasCategorizadas['porcentaje']>0.1, 'Alta', 'Baja')
chi2_contingency(pd.crosstab(index=propinasCategorizadas['sex'],
            columns=propinasCategorizadas['tipoP'], margins=True))

(2.3879980328404988,
 0.66479729597752613,
 4,
 array([[  77.37295082,    9.62704918,   87.        ],
        [ 139.62704918,   17.37295082,  157.        ],
        [ 217.        ,   27.        ,  244.        ]]))

Se puede apreciar que el p-value es 0,99, lo que muestra que la prueba no permite rechazar la hipótesis nula (Existe una relación entre las variables), por lo que no se puede establacer una dependencia entre las variables.

In [9]:
pd.crosstab(index=propinasCategorizadas['tipoP'], columns=propinasCategorizadas['sex']
           ).apply(lambda r: r/r.sum() *100,
                                axis=0)

sex,Female,Male
tipoP,,
Alta,93.103448,86.624204
Baja,6.896552,13.375796


**5. ¿Existe alguna dependencia entre las variables hora del día y propina?**

In [10]:
chi2_contingency(pd.crosstab(index=propinasCategorizadas['time'],
            columns=propinasCategorizadas['tipoP'], margins=True))

(2.5736035698085034,
 0.63150618444844431,
 4,
 array([[ 156.52459016,   19.47540984,  176.        ],
        [  60.47540984,    7.52459016,   68.        ],
        [ 217.        ,   27.        ,  244.        ]]))

In [11]:
pd.crosstab(index=propinasCategorizadas['tipoP'], columns=propinasCategorizadas['time']
           ).apply(lambda r: r/r.sum() *100,
                                axis=0)

time,Dinner,Lunch
tipoP,,
Alta,86.931818,94.117647
Baja,13.068182,5.882353


Con la prueba chi cuadrado se puede ver que el *p-value* es muy alto y no se puede establecer una dependencia entre las variables. Y en la tabla cruzada se puede ver que las propinas se dan muy equitativamente tanto en la noche como al medio día.

**6. ¿Existe relación de dependencia entre las variables de dia de la semana y la propina?**  

In [12]:
pd.crosstab(index=propinasCategorizadas['tipoP'], columns=propinasCategorizadas['day']
           ).apply(lambda r: r/r.sum() *100,
                                axis=0)

day,Fri,Sat,Sun,Thur
tipoP,,,,
Alta,100.0,83.908046,88.157895,93.548387
Baja,0.0,16.091954,11.842105,6.451613


En la tabla cruzada se puede ver que el restaurante sólo abre de jueves a domingo, además que las propinas son similares en todos los días, es decir sin importar que día de la semana sea la propina alta se da entre el 52% y 58% de los casos.

In [13]:
chi2_contingency(pd.crosstab(index=propinasCategorizadas['day'],
            columns=propinasCategorizadas['tipoP'], margins=True))

(5.9853305337585487,
 0.64887517073182144,
 8,
 array([[  16.89754098,    2.10245902,   19.        ],
        [  77.37295082,    9.62704918,   87.        ],
        [  67.59016393,    8.40983607,   76.        ],
        [  55.13934426,    6.86065574,   62.        ],
        [ 217.        ,   27.        ,  244.        ]]))

Igual que en el caso anterior, el *p-value* es muy alto, por lo que la hipótesis nula no se rechaza y no se establece una dependencia entre el día de la semana y el tipo de propina.